# Урок 6. Задача look-alike

## На основе датасета из ноутбука проведите анализ данных и постройте зависимости частоты и среднего убытка от какого-либо параметра (за исключением уже выполненных в ноутбуке).

In [36]:
# Подключаем библиотеки

import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

In [12]:
df = pd.read_csv('freMPL-R.csv', low_memory = False)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343080 entries, 0 to 343079
Data columns (total 31 columns):
Exposure             343080 non-null float64
LicAge               343080 non-null int64
RecordBeg            343080 non-null object
RecordEnd            181115 non-null object
VehAge               177880 non-null object
Gender               343080 non-null object
MariStat             343080 non-null object
SocioCateg           343080 non-null object
VehUsage             343080 non-null object
DrivAge              343080 non-null int64
HasKmLimit           343080 non-null int64
BonusMalus           343080 non-null int64
VehBody              145780 non-null object
VehPrice             145780 non-null object
VehEngine            145780 non-null object
VehEnergy            145780 non-null object
VehMaxSpeed          145780 non-null object
VehClass             145780 non-null object
ClaimAmount          343080 non-null float64
RiskVar              145780 non-null float64
Garage    

In [14]:
df.head()

,Exposure,LicAge,RecordBeg,RecordEnd,VehAge,Gender,MariStat,SocioCateg,VehUsage,DrivAge,...,ClaimInd,Dataset,DeducType,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea
0,0.583,366,2004-06-01,NaN,2,Female,Other,CSP1,Professional,55,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.200,187,2004-10-19,NaN,0,Male,Alone,CSP55,Private+trip to office,34,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.083,169,2004-07-16,2004-08-16,1,Female,Other,CSP1,Professional,33,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.375,170,2004-08-16,NaN,1,Female,Other,CSP1,Professional,34,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.500,224,2004-01-01,2004-07-01,3,Male,Other,CSP47,Professional,53,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# удаляем дубликаты
print(f"With duplicates: {len(df)}")
df = df.drop_duplicates()

With duplicates: 343080


In [16]:
# Смотрим, какие факторы пропущены в каждом из датасетов

dct = {}
for i in range(1,11):
    _x = df.loc[df.Dataset == i].notnull().sum()
    dct[i] = list(_x[_x == 0].index)

print('Dataset Missing Variables')
for x in range(1,11):
    print(x,'\t',dct[x])

Dataset Missing Variables
1 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
2 	 ['DeducType', 'ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
3 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
4 	 ['ClaimNbResp', 'ClaimNbNonResp', 'ClaimNbParking', 'ClaimNbFireTheft', 'ClaimNbWindscreen', 'OutUseNb', 'RiskArea']
5 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
6 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
7 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass', 'RiskVar', 'Garage', 'DeducType']
8 	 ['VehAge', 'VehBody', 'VehPrice', 'VehEngine', 'VehEnergy', 'VehMaxSpeed', 'VehClass'

In [17]:
# Наборы данных 1-4 не содержат информации о количестве страховых требований
# Объединяем наборы данных 5-9, удаляем пустые столбцы, удаляем дубликаты

df_5_9 = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df_5_9 = df_5_9.drop(['Dataset'], axis = 1)
df_5_9 = df_5_9.dropna(axis = 1, how = 'all')
print(f"Without duplicates: {len(df_5_9)}")

Without duplicates: 115190


In [18]:
df_5_9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115190 entries, 145780 to 310979
Data columns (total 20 columns):
Exposure             115190 non-null float64
LicAge               115190 non-null int64
RecordBeg            115190 non-null object
RecordEnd            59467 non-null object
Gender               115190 non-null object
MariStat             115190 non-null object
SocioCateg           115190 non-null object
VehUsage             115190 non-null object
DrivAge              115190 non-null int64
HasKmLimit           115190 non-null int64
BonusMalus           115190 non-null int64
ClaimAmount          115190 non-null float64
ClaimInd             115190 non-null int64
ClaimNbResp          115190 non-null float64
ClaimNbNonResp       115190 non-null float64
ClaimNbParking       115190 non-null float64
ClaimNbFireTheft     115190 non-null float64
ClaimNbWindscreen    115190 non-null float64
OutUseNb             115190 non-null float64
RiskArea             115190 non-null float64
d

In [19]:
df_5_9['PolicyCount'] = 1
df_5_9['ClaimCount'] = df_5_9['ClaimAmount'] > 0
df_5_9['NoClaimCount'] = df_5_9.PolicyCount - df_5_9.ClaimCount
df_5_9[['PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].head()

,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
145780,1,0.0,False,1
145781,1,0.0,False,1
145782,1,0.0,False,1
145783,1,0.0,False,1
145784,1,0.0,False,1


In [33]:
df_group_bm = df_5_9[['Exposure', 'PolicyCount','ClaimAmount', 'ClaimCount', 'NoClaimCount']].groupby(df['BonusMalus']).sum()
df_group_bm = df_group_bm.reset_index()

In [34]:
df_group_bm.head()

,BonusMalus,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount
0,50,30675.653,68026,1.112703e+07,5831.0,62195
1,51,953.531,2218,3.017447e+05,184.0,2034
2,52,386.436,915,2.110474e+05,83.0,832
3,53,240.039,551,1.251411e+05,64.0,487
4,54,1030.683,2413,3.602272e+05,225.0,2188


In [35]:
fig = px.bar(df_group_bm, x = 'BonusMalus', y = 'PolicyCount', title = 'Распределение числа полисов в зависимости от бонус-малуса')
fig.show()

In [41]:
fig = go.Figure(data=[go.Bar(name = 'No Claims', x=df_group_bm.BonusMalus, y=df_group_bm.NoClaimCount),
                      go.Bar(name = 'With Claims', x=df_group_bm.BonusMalus, y=df_group_bm.ClaimCount)])
fig.update_layout(barmode = 'stack', xaxis_title = 'BonusMalus', yaxis_title = 'PolicyCount',title = 'Распределение числа полисов по бонус-малусу с убытками и без')
fig.show()

In [38]:
df_group_bm['Freq'] = df_group_bm.ClaimCount / df_group_bm.Exposure
df_group_bm['AvgClaim'] = df_group_bm.ClaimAmount / df_group_bm.ClaimCount

In [39]:
df_group_bm.head()

,BonusMalus,Exposure,PolicyCount,ClaimAmount,ClaimCount,NoClaimCount,Freq,AvgClaim
0,50,30675.653,68026,1.112703e+07,5831.0,62195,0.190086,1908.253563
1,51,953.531,2218,3.017447e+05,184.0,2034,0.192967,1639.916702
2,52,386.436,915,2.110474e+05,83.0,832,0.214783,2542.739242
3,53,240.039,551,1.251411e+05,64.0,487,0.266623,1955.329362
4,54,1030.683,2413,3.602272e+05,225.0,2188,0.218302,1601.009934


In [42]:
fig = px.bar(df_group_bm, x = 'BonusMalus', y = 'Freq', title = 'Зависимость частоты убытков от бонус-малуса')
fig.show()

In [43]:
fig = px.bar(df_group_bm, x = 'BonusMalus', y = 'AvgClaim', title = 'Зависимость среднего убытка от БМ')
fig.show()

На самом деле судя по графику, можно выдвинуть много предположений, почему частота убытков растет. Тут скорее надо дополнительное обследование, т.к. это может быть и выброс (самый большой), может аномалия, которая была.